In [2]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm 
from transformers import ViTForImageClassification
from utils import get_dataloaders, set_all_seeds
from dataclasses import dataclass, field
from typing import Dict
from IPython.display import HTML

In [5]:
!ls datasets

APTOS  DDR  FGADR  IDRiD  Messidor


In [36]:
@dataclass
class DataConfig:
    dir: str
    batch_size: int
    num_workers: int

@dataclass
class Config:
    train: DataConfig
    val: DataConfig
    test: DataConfig


config = Config(
    train=DataConfig(
        dir='datasets/DDR/train',
        batch_size=64,
        num_workers=8
    ),
    val=DataConfig(
        dir='datasets/DDR/valid',
        batch_size=64,
        num_workers=8
    ),
    test=DataConfig(
        dir='datasets/DDR/test',
        batch_size=64,
        num_workers=8
    )
)

print(config)

Config(train=DataConfig(dir='datasets/DDR/train', batch_size=64, num_workers=8), val=DataConfig(dir='datasets/DDR/valid', batch_size=64, num_workers=8), test=DataConfig(dir='datasets/DDR/test', batch_size=64, num_workers=8))


In [42]:
ds_names = ['APTOS', 'DDR', 'FGADR', 'IDRiD', 'Messidor']
loaders_lst = []
ds_names_lst = [] 

for ds in ds_names: 
    
    config.train.dir = f'datasets/{ds}/train'
    config.val.dir = f'datasets/{ds}/valid'
    config.test.dir = f'datasets/{ds}/test'
    
    train_loader, valid_loader, test_loader = get_dataloaders(config)
    
    loaders_lst.append(train_loader)
    loaders_lst.append(valid_loader)
    loaders_lst.append(test_loader)
    
    ds_names_lst.append(f'{ds} Train')
    ds_names_lst.append(f'{ds} Validation')
    ds_names_lst.append(f'{ds} Test')

In [43]:
ds_names_lst

['APTOS Train',
 'APTOS Validation',
 'APTOS Test',
 'DDR Train',
 'DDR Validation',
 'DDR Test',
 'FGADR Train',
 'FGADR Validation',
 'FGADR Test',
 'IDRiD Train',
 'IDRiD Validation',
 'IDRiD Test',
 'Messidor Train',
 'Messidor Validation',
 'Messidor Test']

In [44]:
model = ViTForImageClassification.from_pretrained('google/vit-large-patch16-224')
model = model.to('cuda')
model.load_state_dict(torch.load('checkpoints/model_epoch_2_acc_0.815.pth')['model_state_dict'])

<All keys matched successfully>

In [48]:
train_acc = []
train_loss = []

valid_acc = []
valid_loss = []

test_acc = []
test_loss = []

In [49]:
for i , (loader, ds_name) in enumerate(zip(loaders_lst, ds_names_lst)): 
    with torch.inference_mode():
        acc = 0
        avg_loss = 0
        for batch_idx, batch in tqdm(enumerate(loader), total=len(loader)): 

            X, y = batch
            X = X.to(model.device)
            y = y.to(model.device)

            pred = model(pixel_values = X, labels = y)
            class_pred = pred.logits.argmax(axis = 1)
            acc += (class_pred == y).type(torch.float).mean()
            avg_loss += pred.loss.item()

            del pred
            torch.cuda.empty_cache()


        print(f'Results of model inference on {ds_name}')
        print(f'Loss : {avg_loss / len(loader):.3f}')
        print(f'Accuracy : {acc / len(loader):.3f}')   
        print()
        print()
        
        if (i % 3) == 0 :
            train_acc.append(acc / len(loader))
            train_loss.append(avg_loss / len(loader))
        if (i % 3) == 1 :
            valid_acc.append(acc / len(loader))
            valid_loss.append(avg_loss / len(loader))            
        if (i % 3) == 2 :
            test_acc.append(acc / len(loader))
            test_loss.append(avg_loss / len(loader))            

100%|███████████████████████████████████████████| 49/49 [00:28<00:00,  1.74it/s]

Results of model inference on APTOS Train
Loss : 1.174
Accuracy : 0.589





100%|█████████████████████████████████████████████| 9/9 [00:05<00:00,  1.52it/s]

Results of model inference on APTOS Validation
Loss : 1.147
Accuracy : 0.566





100%|███████████████████████████████████████████| 31/31 [00:17<00:00,  1.75it/s]

...

Results of model inference on Messidor Train
Loss : 1.257
Accuracy : 0.499





100%|█████████████████████████████████████████████| 5/5 [00:04<00:00,  1.22it/s]

Results of model inference on Messidor Validation
Loss : 1.649
Accuracy : 0.375





100%|█████████████████████████████████████████████| 5/5 [00:03<00:00,  1.26it/s]

Results of model inference on Messidor Test
Loss : 1.501
Accuracy : 0.359




In [75]:
df_acc = pd.DataFrame(
{
'Dataset' : ds_names,
'Train split' : [ np.round(item.item(), 3) for item in train_acc],
'Val split' : [ np.round(item.item(), 3) for item in valid_acc],
'Test split' : [ np.round(item.item(), 3) for item in test_acc]
}
)

In [76]:
HTML(df_acc.to_html(index=False))

Dataset,Train split,Val split,Test split
APTOS,0.589,0.566,0.121
DDR,0.669,0.815,0.699
FGADR,0.187,0.281,0.244
IDRiD,0.482,0.516,0.373
Messidor,0.499,0.375,0.359


In [71]:
df_loss = pd.DataFrame(
{
'Dataset' : ds_names,
'Train split' : [ np.round(item, 3) for item in train_loss],
'Val split' : [ np.round(item, 3) for item in valid_loss],
'Test split' : [ np.round(item, 3) for item in test_loss]
}
)

In [72]:
HTML(df_loss.to_html(index=False))

Dataset,Train split,Val split,Test split
APTOS,1.174,1.147,3.504
DDR,0.859,0.638,0.817
FGADR,2.292,1.763,1.846
IDRiD,1.495,1.083,1.599
Messidor,1.257,1.649,1.501
